Mounting Drive to save model and read files

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/MyDrive/Colab Notebooks/Assignment3


/content/drive/MyDrive/Colab Notebooks/Assignment3


In [7]:
!ls

'AOMML Assignment3.ipynb'   model_84_9.pickle		 model_87_2000.pickle
 Dataset.csv		    model_87_2000_later.pickle	 model_fin.pickle


In [8]:
import numpy as np

import os
import csv
from keras.datasets import imdb
from sklearn.metrics import log_loss
from scipy.special import expit

**Loading the Dataset**

In [9]:
num_words= 100


In [10]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=num_words)
# print(testing_data.shape)
# data = np.concatenate((training_data, testing_data), axis=0)
# targets = np.concatenate((training_targets, testing_targets), axis=0)




17473536/17464789 [==============================] - 0s 0us/step


In [11]:
def vectorize(sequences, dimension = 5000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
train_data = vectorize(training_data,dimension=num_words)
train_targets = np.array(training_targets).astype("float32")
testing_data = vectorize(testing_data,dimension=num_words)
testing_targets = np.array(testing_targets).astype("float32")



**Splitting into train and test**

In [12]:

from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(data,targets,shuffle=True,stratify=targets,random_state=42,test_size=0.1)
x_train,x_val,y_train,y_val = train_test_split(train_data,train_targets,shuffle=True,stratify=train_targets,random_state=42,test_size=0.1)




In [13]:
x_test=testing_data
y_test=testing_targets

In [14]:
#COnverting from Nxf to fXN where f is number of features and N is number of samples
x_train = x_train.T
y_train=y_train
x_test = x_test.T
y_test=y_test
x_val=x_val.T
y_val=y_val


**ANN Code**

In [15]:
import copy
from sklearn.utils import shuffle
def accuracy(y_pred,y_true):
  return y_pred[y_pred==y_true].shape[0]/y_pred.shape[0]

class ANN:
  def __init__(self,input_neurons=10000,output_neurons=1,hidden_neurons=[8,8],hidden_activation='sigmoid'):
    self.layers=[input_neurons]
    np.random.seed(0)
    for _ in range(len(hidden_neurons)):
      self.layers.append(hidden_neurons[_]) # adding bias in each layer
    self.layers.append(output_neurons)
    self.w=[] # struct for storing weights of each layer
    self.b=[]
    self.buff_z=[] # buffers to store intermediate activations and pre-activations for gradient computation
    self.buff_a=[]
    self.hidden_activation=hidden_activation
    self.err_buff=[[0]*neurons for neurons in self.layers] # buffer for storing gradient error for each neuron of the network for backprop
    for n in range(len(self.layers)-1):
      # random init of weights and zero init of biases
      w_l=np.random.randn(self.layers[n+1],self.layers[n])*0.1
      b_l=np.zeros(self.layers[n+1])
      # b_l=np.ones(self.layers[n+1])
      self.w.append(w_l)
      self.b.append(b_l)
    

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def activation(self,x,last=False):
    if last:
      # Last layer is fixed to sigmoid
      return self.sigmoid(x)
    if self.hidden_activation == 'sigmoid':
      return 1/(1+np.exp(-x))
    elif self.hidden_activation=='relu':
      xr=np.copy(x).flatten()
      xr[xr<=0] = 0
      return xr.reshape(x.shape)
  def forward(self,X,train=True):
    # Z=np.zeros((self.layers[-1],X.shape[1]))
    Z=copy.deepcopy(X)
    if train:
      self.buff_a=[]
      self.buff_z=[]
      self.buff_a.append(Z) # storing input as the first activation
      self.buff_z.append(0)
    for i in range(0,len(self.layers)-1):
      # Z=np.vstack((np.ones((1,Z.shape[1])),Z))
      w=self.w[i]
      b=np.dstack([self.b[i]]*X.shape[1]).squeeze(0)
      in_shape=Z.shape
      Z = np.matmul(w,Z) + b
      if train: self.buff_z.append(Z)
      if i== len(self.layers)-2:
        Z = self.activation(Z,last=True)
      else:
        Z= self.activation(Z)      
      if train: self.buff_a.append(Z) 
    return Z


  def dz(self,a,y):
    # dC/dZ -- dC/dA * dA/dZ (for BCE loss and sigmoid activation)
    return ( a-y ) / y.shape[0]

  def fdash_a(self,a):
    if self.hidden_activation == 'sigmoid':
      return a*(1-a)
    elif self.hidden_activation =='relu':
      ac= np.copy(a)
      ac=ac.flatten()
      return (np.where(ac>0,1,0)).reshape(a.shape)
      
  def loss(self,a,y_train):
    return log_loss(y_train.flatten(),a.flatten())
  
  def compute_error(self,y):
    last_layer=self.dz(self.buff_a[-1],y)
    err_buff=[0]*(len(self.layers)-1)
    err_buff[-1]=last_layer # [_,_,_,del3]
    for l in range(len(self.layers)-2,0,-1):
      err=np.matmul(self.w[l].T,err_buff[l]) * self.fdash_a(self.buff_a[l+1])
      err_buff[l-1] =err
    return err_buff

  def backprop(self,y_train,lr):
    n=y_train.shape[0]
    err=self.compute_error(y_train)
    self.dw=[]
    self.db=[]
    for i in range(0,len(self.layers)-1):
      # print(err[i].shape,self.buff_a[i].shape)
      self.dw.append(np.matmul(err[i],self.buff_a[i].T)/n)
      self.db.append(err[i].mean(axis=1))


  def fit(self,x_train,y_train,x_val,y_val,epochs=100,lr=0.1,batch_size=4):
    n=x_train.shape[1]
    # print(x_train.shape,y_train.shape)
    for _ in range(epochs):
      train_loss=0
      train_accuracy=0
      x_train,y_train=shuffle(copy.deepcopy(x_train.T),copy.deepcopy(y_train.T))
      x_train=x_train.T
      y_train=y_train.T
      for batch_idx in range(0,n,batch_size):
        x_batch,y_batch=x_train[:,batch_idx:batch_idx+batch_size],y_train[batch_idx:batch_idx+batch_size]
        self.forward(x_batch)
        train_loss += self.loss(self.buff_a[-1],y_batch) # calculate loss with the last activation
        train_pred= self.float_to_bin(self.buff_a[-1])
        train_accuracy += self.accuracy(train_pred,y_batch)
        self.backprop(y_batch,lr)
        for i in range(len(self.w)):
          self.w[i] -=lr*self.dw[i]
          self.b[i] -=lr*self.db[i]
      
      
      y_pred=self.float_to_bin(self.forward(x_val,train=False))
      print(f"Epoch {_}: Training Loss : {train_loss/(x_train.shape[1]/batch_size)}, Train Acc:{train_accuracy/(x_train.shape[1]/batch_size)} Val Acc : {self.accuracy(y_pred,y_val)}")


  def float_to_bin(self,y):
    # converts floats of final layer to predictions {0,1}
    return np.where(y.flatten()>=0.5,0,1)

    # pass
  def predict(self,x_test):
    y_pred=self.forward(x_test,train=False)
    y_pred= self.float_to_bin(y_pred)
    return y_pred

  # def load(weights,bias):

  def accuracy(self,y_pred,y_test):
    y_pred=y_pred.flatten()
    return np.sum(np.abs(y_pred-y_test))/y_pred.shape[0]



In [16]:
model=ANN(input_neurons=x_train.shape[0],hidden_activation='relu')

In [17]:
model.fit(x_train,y_train,x_val,y_val,epochs=50,batch_size=128,lr=1)



Epoch 0: Training Loss : 0.6940340700468027, Train Acc:0.4934826666666667 Val Acc : 0.4844
Epoch 1: Training Loss : 0.694025621495679, Train Acc:0.5017208888888889 Val Acc : 0.4844
Epoch 2: Training Loss : 0.6940186277336483, Train Acc:0.5051111111111111 Val Acc : 0.4872
Epoch 3: Training Loss : 0.6940115271328638, Train Acc:0.5003555555555556 Val Acc : 0.4912
Epoch 4: Training Loss : 0.6940044949219311, Train Acc:0.5044462222222222 Val Acc : 0.4964
Epoch 5: Training Loss : 0.6939941066155109, Train Acc:0.5119751111111112 Val Acc : 0.4964
Epoch 6: Training Loss : 0.6939856316839219, Train Acc:0.511632 Val Acc : 0.5008
Epoch 7: Training Loss : 0.6939768600880649, Train Acc:0.5147235555555556 Val Acc : 0.5044
Epoch 8: Training Loss : 0.6939699852940542, Train Acc:0.5143928888888889 Val Acc : 0.4944
Epoch 9: Training Loss : 0.6939604335780084, Train Acc:0.509296 Val Acc : 0.5036
Epoch 10: Training Loss : 0.6939527678972431, Train Acc:0.5129528888888889 Val Acc : 0.5044
Epoch 11: Training 

Test Accuracy

In [ ]:
y_pred= model.predict(x_test)
print(model.accuracy(y_pred,y_test))

0.70536


Saving and loading Model 

In [ ]:
import pickle
with open('model_fin.pickle','wb') as file:
  pickle.dump(model,file)

In [ ]:
#@title
import pickle
with open('model_fin.pickle','rb') as file2:
  model_load=pickle.load(file2)


In [ ]:
y_pred= model_load.predict(x_test)
print(model_load.accuracy(y_pred,y_test))

0.86632
